In [2]:
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes
!pip install plotly-geo
!pip install -c plotly plotly-geo
!pip install plotly-geo
!pip install plotly
from urllib.request import urlopen

import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from plotly import graph_objects

import plotly.figure_factory as ff

import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)



     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 278kB/s 
     |████████████████████████████████| 10.9MB 43.7MB/s 
     |████████████████████████████████| 225kB 2.8MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.0-cp36-none-any.whl size=32609 sha256=aabe5ffda338af7e1545fd1fe858749ce47ef493153201a114ec50aaf6078f79
  Stored in directory: /root/.cache/pip/wheels/a6/0c/de/321b5192ad416b328975a2f0385f72c64db4656501eba7cc1a
Successfully built pyshp
Requirement already up-to-date: shapely in /usr/local/lib/python3.6/dist-packages (1.7.0)
Requirement already up-to-date: descartes in /usr/local/lib/python3.6/dist-packages (1.1.0)
     |████████████████████████████████| 23.7MB 1.3MB/s 
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'plotly'


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/magnesium-nitrate/election-visuals-2016/master/genelc2016.csv',dtype={"FIPS":str})

for value, row in df.iterrows():
    review = df['FIPS'][value]
    
    #analysis = TextBlob(str(review))
    #yeet = analysis.sentiment
    if(len(review)==4):
      df.at[value,'FIPS'] = ("0"+review)

import plotly.express as px
df2 = pd.DataFrame({'party':['republican'], 'winner':['Donald Trump'],'FIPS':[10345],'totalvotes':[32813],'candidatevotes':[20083],'county+st':["abc,nc"]})
#temp_df = {'party': "democrat", 'winner': 'Hillary Clinton','FIPS':10347,'totalvotes':31233,'candidatevotes':25023}


#temp_df = {'party': 10, 'winner': 11, 'FIPS': 12,'totalvotes':123,'candidatevotes':1239}
#df = df2.append(temp_df, ignore_index=True)
#print(df)
current = 0;
dem_temp, rep_temp, other_temp = 0,0,0
#df2.append(temp_df,ignore_index=True);
for value, row in df.iterrows():

    current_fips = df['FIPS'][value]
    if(current!=current_fips):
      current = current_fips
      dem_temp = df['candidatevotes'][value]
    else:
      if(df['party'][value]=='republican'):
        rep_temp = df['candidatevotes'][value]
      else:
        other_temp = df['candidatevotes'][value]
        if(dem_temp>rep_temp):
          if(df['totalvotes'][value]-other_temp!=0):
            c = 1/(df['totalvotes'][value]-other_temp)
            temp = {"party":"democrat","winner":"Hillary Clinton","FIPS":df['FIPS'][value],"totalvotes":df['totalvotes'][value]-other_temp,"candidatevotes":(dem_temp)*c,"county+st":df['county'][value]+","+df['state'][value]}
            df2 = df2.append(temp,ignore_index=True)
        else:
          if(df['totalvotes'][value]-other_temp!=0):
            c = 1/(df['totalvotes'][value]-other_temp)
            temp = {"party":"republican","winner":"Donald Trump","FIPS":df['FIPS'][value],"totalvotes":df['totalvotes'][value]-other_temp,"candidatevotes":1-(rep_temp)*c,"county+st":df['county'][value]+","+df['state'][value]}
            df2 = df2.append(temp,ignore_index=True)
    
    #analysis = TextBlob(str(review))
    #yeet = analysis.sentiment
    """
    if(review=='democrat'):
      if(df['totalvotes'][value]!=0):
        c = 1/(df['totalvotes'][value])
        if(df['totalvotes'][value]-df['candidatevotes'][value]>df['candidatevotes'][value]):
          temp = {"party":"republican","winner":"Donald Trump","FIPS":df['FIPS'][value],"totalvotes":df['totalvotes'][value],"candidatevotes":1-(df['totalvotes'][value]-df['candidatevotes'][value])*c,"county+st":df['county'][value]+","+df['state'][value]}
          #print(temp)
          df2 = df2.append(temp,ignore_index=True)
        else:
          temp = {"party":"democrat","winner":"Hillary Clinton","FIPS":df['FIPS'][value],"totalvotes":df['totalvotes'][value],"candidatevotes":(df['candidatevotes'][value])*c,"county+st":df['county'][value]+","+df['state'][value]}
          df2 = df2.append(temp,ignore_index=True)
      """

fig = px.choropleth_mapbox(df2, geojson=counties, locations='FIPS', color='candidatevotes',
                           color_continuous_scale="Bluered_r",
                           range_color=(0, 1),                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'candidatevotes':'percentage d/r'},hover_data=["totalvotes","winner","county+st"]
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()